In [39]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
import os
import requests
from pathlib import Path
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from MCForecastTools import MCSimulation

In [50]:
# Import API data
from dotenv import load_dotenv
import alpaca_trade_api as tradeapi

In [51]:
# Load .env enviroment variables
load_dotenv("ProjectAPI.env")

True

In [52]:
# Set the tickers
tickers = ["SPY", "IWM", "EZU", "EWU", "EWJ", "EWZ", "EWA", "EWC", "MCHI", "EWW"]

# Create the shares DataFrame
df_etf = pd.DataFrame(index=tickers)

In [53]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("Alpaca_API_Key")
alpaca_secret_key = os.getenv("Alpaca_Secret_Key")

# Verify that Alpaca key and secret were correctly loaded
print(f"Alpaca Key type: {type(alpaca_api_key)}")
print(f"Alpaca Secret Key type: {type(alpaca_secret_key)}")

Alpaca Key type: <class 'str'>
Alpaca Secret Key type: <class 'str'>


In [54]:
# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [55]:
# Format current date as ISO format
# Set start and end datetimes of 18 months
start_date = pd.Timestamp("2022-04-01", tz="America/New_York").isoformat()
end_date = pd.Timestamp("2023-10-01", tz="America/New_York").isoformat()

In [56]:
# Set timeframe to 18 months for the Alpaca API
timeframe = "1Day"

In [71]:
#Get 18 months of pricing data for ETF portfolio
df_portfolio = alpaca.get_bars(tickers, timeframe, start=start_date, end=end_date).df

df_portfolio

,close,high,low,trade_count,open,volume,vwap,symbol
timestamp,,,,,,,,
2022-04-01 04:00:00+00:00,26.69,26.7000,26.475,7069,26.61,4285764,26.579325,EWA
2022-04-04 04:00:00+00:00,26.88,26.9050,26.630,7695,26.69,3288300,26.806707,EWA
2022-04-05 04:00:00+00:00,26.74,27.1600,26.650,11594,27.00,6864193,26.829930,EWA
2022-04-06 04:00:00+00:00,26.56,26.7450,26.410,10866,26.71,3946883,26.574937,EWA
2022-04-07 04:00:00+00:00,26.53,26.6100,26.290,9064,26.44,4980263,26.453717,EWA
...,...,...,...,...,...,...,...,...
2023-09-25 04:00:00+00:00,432.23,432.2700,428.720,488035,429.17,70874824,430.900652,SPY
2023-09-26 04:00:00+00:00,425.88,429.8200,425.020,633690,429.09,96172890,427.150722,SPY
2023-09-27 04:00:00+00:00,426.05,427.6700,422.290,693420,427.09,104707452,425.420576,SPY


In [72]:
#Drop unnecessary columns for our portfolio dataframe
df_portfolio.drop(columns=['high','low','open','trade_count','vwap'], axis=1, inplace=True)

#Rearrange columns
df_portfolio = df_portfolio[['symbol', 'close', 'volume']]

df_portfolio

,symbol,close,volume
timestamp,,,
2022-04-01 04:00:00+00:00,EWA,26.69,4285764
2022-04-04 04:00:00+00:00,EWA,26.88,3288300
2022-04-05 04:00:00+00:00,EWA,26.74,6864193
2022-04-06 04:00:00+00:00,EWA,26.56,3946883
2022-04-07 04:00:00+00:00,EWA,26.53,4980263
...,...,...,...
2023-09-25 04:00:00+00:00,SPY,432.23,70874824
2023-09-26 04:00:00+00:00,SPY,425.88,96172890
2023-09-27 04:00:00+00:00,SPY,426.05,104707452


In [80]:
# Reorganize the DataFrame and separate ticker data
# Order:  SPY, EWA, EWC, EWJ, EWU, EWZ, EWW, EZU, IWM, MCHI

# Separate ticker data
SPY = df_portfolio[df_portfolio["symbol"]=="SPY"].drop("symbol", axis=1)
EWA = df_portfolio[df_portfolio["symbol"]=="EWA"].drop("symbol", axis=1)
EWC = df_portfolio[df_portfolio["symbol"]=="EWC"].drop("symbol", axis=1)
EWJ = df_portfolio[df_portfolio["symbol"]=="EWJ"].drop("symbol", axis=1)
EWU = df_portfolio[df_portfolio["symbol"]=="EWU"].drop("symbol", axis=1)
EWZ = df_portfolio[df_portfolio["symbol"]=="EWZ"].drop("symbol", axis=1)
EWW = df_portfolio[df_portfolio["symbol"]=="EWW"].drop("symbol", axis=1)
EZU = df_portfolio[df_portfolio["symbol"]=="EZU"].drop("symbol", axis=1)
IWM = df_portfolio[df_portfolio["symbol"]=="IWM"].drop("symbol", axis=1)
MCHI = df_portfolio[df_portfolio["symbol"]=="MCHI"].drop("symbol", axis=1)

# Concatenate the ticker DataFrames
df_portreo = pd.concat([SPY, EWA, EWC, EWJ, EWU, EWZ, EWW, EZU, IWM, MCHI], axis=1)

# Rename columns
df_portreo.columns=['SPY_close', 'SPY_volume', 'EWA_close', 'EWA_volume', 'EWC_close', 'EWC_volume', 'EWJ_close', 'EWJ_volume', 'EWU_close', 'EWU_volume', 'EWZ_close', 'EWZ_volume', 'EWW_close', 'EWW_volume', 'EZU_close', 'EZU_volume', 'IWM_close', 'IWM_volume', 'MCHI_close', 'MCHI_volume']

# Display sample data
df_portreo

,SPY_close,SPY_volume,EWA_close,EWA_volume,EWC_close,EWC_volume,EWJ_close,EWJ_volume,EWU_close,EWU_volume,EWZ_close,EWZ_volume,EWW_close,EWW_volume,EZU_close,EZU_volume,IWM_close,IWM_volume,MCHI_close,MCHI_volume
timestamp,,,,,,,,,,,,,,,,,,,,
2022-04-01 04:00:00+00:00,452.92,90526299,26.69,4285764,40.22,4957883,61.93,5887065,33.93,2991891,39.14,32915414,55.36,1798341,43.66,5484519,207.57,26893201,54.75,7382913
2022-04-04 04:00:00+00:00,456.80,61390954,26.88,3288300,40.62,6214440,62.43,5316176,33.90,2491503,39.52,21460222,55.07,3232716,43.67,3563681,207.91,21021131,56.49,7068617
2022-04-05 04:00:00+00:00,451.03,75760010,26.74,6864193,40.33,8828963,60.80,5570811,33.95,6544095,38.32,30567939,54.14,2301442,42.88,10249067,202.99,27050940,54.92,8027718
2022-04-06 04:00:00+00:00,446.52,107084639,26.56,3946883,39.90,5466978,59.89,6255074,33.99,4946990,37.67,44208762,53.61,3050678,42.04,9638908,200.11,34969069,54.02,7041040
2022-04-07 04:00:00+00:00,448.77,78235837,26.53,4980263,39.79,4362696,59.68,5599909,34.14,3988989,37.77,24694010,53.43,1793140,42.16,7227281,199.46,29692911,53.07,7808268
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-09-25 04:00:00+00:00,432.23,70874824,21.62,1974029,34.10,1229511,61.72,6078325,31.85,2399454,31.00,20674978,58.83,2105042,42.42,23485612,177.46,22416152,43.41,5561411
2023-09-26 04:00:00+00:00,425.88,96172890,21.34,2408206,33.56,1609404,60.77,8765694,31.55,2252518,30.43,20042322,58.17,2054052,41.80,4721707,174.36,30476036,42.91,3113411
2023-09-27 04:00:00+00:00,426.05,104707452,21.25,2527763,33.39,1311190,61.03,8211960,31.50,2305536,30.06,30559237,57.96,2409395,41.66,5250708,175.99,37295094,43.05,3678446


In [92]:
SPY_mc = pd.concat([SPY,EWA], axis=1, keys=['SPY','EWA'])

SPY_mc

SPY               EWA         
                            close     volume  close   volume
timestamp                                                   
2022-04-01 04:00:00+00:00  452.92   90526299  26.69  4285764
2022-04-04 04:00:00+00:00  456.80   61390954  26.88  3288300
2022-04-05 04:00:00+00:00  451.03   75760010  26.74  6864193
2022-04-06 04:00:00+00:00  446.52  107084639  26.56  3946883
2022-04-07 04:00:00+00:00  448.77   78235837  26.53  4980263
...                           ...        ...    ...      ...
2023-09-25 04:00:00+00:00  432.23   70874824  21.62  1974029
2023-09-26 04:00:00+00:00  425.88   96172890  21.34  2408206
2023-09-27 04:00:00+00:00  426.05  104707452  21.25  2527763
2023-09-28 04:00:00+00:00  428.52   92258310  21.63  1440039
2023-09-29 04:00:00+00:00  427.48  115111320  21.51  1635771

[376 rows x 4 columns]

In [81]:
#Export clean dataframe to .csv file for backup
#df_portfolio.to_csv('UpDownFlat.csv')

df_portreo.to_csv(r'Desktop\UpDownFlatData.csv')

In [96]:
#Create the Monte Carlo Simulation for the next day
MC_df = SPY_mc[["SPY"]]

monte_predict = MCSimulation(
    portfolio_data = MC_df,
    num_simulation = 1000,
    num_trading_days = 1
)

#Check it worked
monte_predict.portfolio_data.head()

SPY                        
                            close     volume daily_return
timestamp                                                
2022-04-01 04:00:00+00:00  452.92   90526299          NaN
2022-04-04 04:00:00+00:00  456.80   61390954     0.008567
2022-04-05 04:00:00+00:00  451.03   75760010    -0.012631
2022-04-06 04:00:00+00:00  446.52  107084639    -0.009999
2022-04-07 04:00:00+00:00  448.77   78235837     0.005039

In [97]:
# Run a Monte Carlo simulation to forecast tomorrow's movement
from MCForecastTools import MCSimulation

monte_predict.calc_cumulative_return()

Running Monte Carlo simulation number 0.
Running Monte Carlo simulation number 10.
Running Monte Carlo simulation number 20.
Running Monte Carlo simulation number 30.
Running Monte Carlo simulation number 40.
Running Monte Carlo simulation number 50.
Running Monte Carlo simulation number 60.
Running Monte Carlo simulation number 70.
Running Monte Carlo simulation number 80.
Running Monte Carlo simulation number 90.
Running Monte Carlo simulation number 100.
Running Monte Carlo simulation number 110.
Running Monte Carlo simulation number 120.
Running Monte Carlo simulation number 130.
Running Monte Carlo simulation number 140.
Running Monte Carlo simulation number 150.
Running Monte Carlo simulation number 160.
Running Monte Carlo simulation number 170.
Running Monte Carlo simulation number 180.
Running Monte Carlo simulation number 190.
Running Monte Carlo simulation number 200.
Running Monte Carlo simulation number 210.
Running Monte Carlo simulation number 220.
Running Monte Carlo si

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,1.000000,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.00000,1.000000,...,1.00000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
1,1.011065,0.97888,1.000497,0.993531,1.025818,1.000067,0.980111,1.004702,1.02726,0.995012,...,1.00378,0.986189,1.002613,1.004956,1.001416,1.021099,1.020052,0.960908,0.992464,1.014259


In [98]:
#Show what the prediction average and std are to give an idea of tomorrow's likelyhood of up/down
MC_tbl = monte_predict.summarize_cumulative_return()
MC_tbl

count           1000.000000
mean               1.000185
std                0.012706
min                0.951322
25%                0.991358
50%                1.000413
75%                1.008837
max                1.054788
95% CI Lower       0.976056
95% CI Upper       1.024438
Name: 1, dtype: float64

In [ ]:
#based on the table output, display the mean and 1.5 std up and down to show the target outcomes.


In [ ]:
#define variables for regression

X=df_portfolio[["IWM", "EZU", "EWU", "EWJ", "EWZ", "EWA", "EWC", "MCHI", "EWW"]]

y=df_portfolio['SPY']

In [ ]:
#set up test and train section of the df
X_train,X_test, y_train, y_test = train_test_split(X, y, test_size = 0.1, random_state = 0)

In [ ]:
#run regression and predict
reg = LinearRegression()
 
# Fit the model to the data
reg_model = LinearRegression().fit(X_train, y_train)

# Print the coefficients of the model to determine influences on X
coeffs = reg_model.coef_
ticks = ["IWM", "EZU", "EWU", "EWJ", "EWZ", "EWA", "EWC", "MCHI", "EWW"]
count = 0
for t in coeffs:
    print(f"{ticks[count]} beta is {round(coeffs[count],4)}")
    count +=1

In [ ]:
#Predicting the Test and Train set result 

y_pred= reg_model.predict(X_test)  
x_pred= reg_model.predict(X_train)

#finding the difference between predicted and actual values- for reference of predictive accuracy of model
reg_model_diff = pd.DataFrame({'Actual value': y_test, 'Predicted value': y_pred})
reg_model_diff.plot(figsize=(20,10))

In [ ]:
#display the regression output into the GUI for analysis
